# 12/19 ゼミ  

In [16]:
# import Pkg
# Pkg.add("DataFrames")
# Pkg.add("CSV")
# Pkg.add("IntervalSets")
# Pkg.add("LatexStrings")

In [17]:
using DataFrames, CSV
using IntervalArithmetic
using LaTeXStrings
using Statistics
# using Base
include("./libs/display-latex.jl")
include("./libs/crisp-pcm.jl")
include("./libs/analysis-indicators.jl")
include("./libs/solve-deterministic-ahp.jl")
include("./libs/AMR.jl")

AMR (generic function with 1 method)

In [18]:
# 評価基準数
num_criteria = [8];
# 真の区間重要度のセッティング
true_importance = ["A"]; # ["A", "B", "C", "D", "E"]

In [19]:
# 出力ファイル設定
csv_file_path = "./out/AMR-A-3.csv"

# DataFrameを作成
data = DataFrame(Num=Int[], Setting=String[], P=Float64[], Q=Float64[], R=Float64[], F=Float64[], Dist_ave=Float64[], Dist_std=Float64[])

Row,Num,Setting,P,Q,R,F,Dist_ave,Dist_std
,Int64,String,Float64,Float64,Float64,Float64,Float64,Float64


In [20]:
# 真の区間重要度を読み込み
for N in num_criteria
        
    for setting in true_importance
        df = CSV.File("./data/N"*string(N)*"_"*setting*"_Given_interval_weight.csv", header=false) |> DataFrame
        T = Vector{Interval{Float64}}() # Float64のところにTを使うとおかしくなるのはなぜか？
        for i in 1:N
            push!(T, interval(df[1, 2*i-1], df[1, 2*i]))
        end

        # display(L"T = %$(intervalVectorLaTeXString(T))")
        # 一対比較行列を1000個読み込み
        df1 = CSV.File("./data/N"*string(N)*"_a3_"*setting*"_PCM_int.csv", header=false) |> DataFrame

        # データフレームをN×Nのサブデータフレームに分割
        subdfs = split_dataframe(df1, N);
        # 結果格納用
        P = Matrix{Float64}(undef, N, 1000)
        Q = Matrix{Float64}(undef, N, 1000)
        R = Matrix{Float64}(undef, N, 1000)
        F = Matrix{Float64}(undef, N, 1000)
        Distance = Vector{Float64}(undef, 1000)

        # 1000個のデータで推定
        for (i, subdf) in enumerate(subdfs)

            # 一対比較行列を取り出す
            A = Matrix(subdf)
            # display(L"A = %$(matrixLaTeXString(A))")
            # if i == 779
            #     println("i = ", i)
            #     display(L"A = %$(matrixLaTeXString(A))")
            # end

            # 提案手法で区間重要度推定
            solution = AMR(A, ALD)
            E = solution.W # 区間重要度の推定値
            # display(L"E = %$(intervalVectorLaTeXString(E))")
            # display(L"T = %$(intervalVectorLaTeXString(T))")

            TcapE = T .∩ E # 真の区間重要度と推定値の積集合
            TcupE = T .∪ E # 真の区間重要度と推定値の和集合
            # display(L"T ∩ E = %$(intervalVectorLaTeXString(TcapE))")
            # display(L"T ∪ E = %$(intervalVectorLaTeXString(TcupE))")
            
            # P, Q, R, F の計算
            P[:, i] = calculate_P(T, E) # 真の区間重要度と推定値の積集合の割合
            Q[:, i] = calculate_Q(T, E) # 真の区間重要度と推定値の積集合の割合
            R[:, i] = calculate_R(T, E) # 真の区間重要度と推定値の積集合の割合
            F[:, i] = calculate_F(T, E) # F値
            Distance[i] = calculate_manhattan(interval_centers(T), interval_centers(E))/N
            # println("i = ", i)
            # display(L"P = %$(VectorLaTeXString(P[:,i]))")
            # display(L"Q = %$(VectorLaTeXString(Q[:,i]))")
            # display(L"R = %$(VectorLaTeXString(R[:,i]))")
            # display(L"F = %$(VectorLaTeXString(F[:,i]))")
        end

        println("N = ", N)
        println("setting = ", setting)

        P_ave = mean(P)
        Q_ave = mean(Q)
        R_ave = mean(R)
        F_ave = mean(F)
        Dist_ave = mean(Distance)
        Dist_std = std(Distance)
        # DataFrameにデータを追加
        push!(data, (N, setting, P_ave, Q_ave, R_ave, F_ave, Dist_ave, Dist_std))

        display(L"Pₐᵥₑ = %$(round(P_ave, digits=5))")
        display(L"Qₐᵥₑ = %$(round(Q_ave, digits=5))")
        display(L"Rₐᵥₑ = %$(round(R_ave, digits=5))")
        display(L"Fₐᵥₑ = %$(round(F_ave, digits=5))")
        P_var = var(P)
        Q_var = var(Q)
        R_var = var(R)
        F_var = var(F)
        display(L"Pᵥₐᵣ = %$(round(P_var, digits=5))")
        display(L"Qᵥₐᵣ = %$(round(Q_var, digits=5))")
        display(L"Rᵥₐᵣ = %$(round(R_var, digits=5))")
        display(L"Fᵥₐᵣ = %$(round(F_var, digits=5))")

    end
end

# DataFrameをCSVファイルに書き込み
# CSV.write(csv_file_path, data)


L"$Pₐᵥₑ = 0.63333$"

L"$Qₐᵥₑ = 0.88962$"

L"$Rₐᵥₑ = 0.70886$"

L"$Fₐᵥₑ = 0.76406$"

L"$Pᵥₐᵣ = 0.02386$"

L"$Qᵥₐᵣ = 0.02262$"

L"$Rᵥₐᵣ = 0.03411$"

L"$Fᵥₐᵣ = 0.01497$"

N = 8
setting = A
